<a href="https://colab.research.google.com/github/MujeebDawar/ComputerVisionTask/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO


In [19]:
# Global variable for storing the uploaded grayscale image
uploaded_grayscale_image = None

In [20]:
def upload_grayscale_image():
    """Upload a grayscale image file and store it globally."""
    image_upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
    display(image_upload_widget)

    def on_image_upload_change(change):
        global uploaded_grayscale_image
        uploaded_file_content = list(image_upload_widget.value.values())[0]['content']
        grayscale_image = Image.open(BytesIO(uploaded_file_content)).convert("L")  # Convert to grayscale
        uploaded_grayscale_image = np.array(grayscale_image)
        print("Grayscale image uploaded successfully! Please proceed with transformations.")
        image_upload_widget.close()  # Hide the widget after upload

    image_upload_widget.observe(on_image_upload_change, names='value')

In [24]:
def apply_log_transformation(image, scaling_constant):
    """Applies log transformation to enhance the image."""
    # Convert image to float and normalize
    image = image.astype(np.float32) / 255.0
    # Apply log transformation
    log_transformed_image = scaling_constant * np.log(1 + image)
    # Scale back to the range [0, 255]
    log_transformed_image = np.clip(log_transformed_image * 255, 0, 255).astype(np.uint8)
    return log_transformed_image

In [25]:

def apply_power_law_transformation(image, scaling_constant, gamma_value):
    """Applies power-law (gamma) transformation to enhance the image."""
    # Convert image to float and normalize
    image = image.astype(np.float32) / 255.0
    # Apply power-law transformation
    power_law_transformed_image = scaling_constant * np.power(image, gamma_value)
    # Scale back to the range [0, 255]
    power_law_transformed_image = np.clip(power_law_transformed_image * 255, 0, 255).astype(np.uint8)
    return power_law_transformed_image

In [26]:
def show_transformed_images(log_scaling_constant, power_scaling_constant, gamma_value):
    """Displays the original image and the transformations side by side."""
    global uploaded_grayscale_image
    if uploaded_grayscale_image is None:
        print("Please upload an image first.")
        return

    # Apply the transformations
    log_transformed = apply_log_transformation(uploaded_grayscale_image, log_scaling_constant)
    power_law_transformed = apply_power_law_transformation(uploaded_grayscale_image, power_scaling_constant, gamma_value)

    # Display the original and enhanced images
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    ax[0].imshow(uploaded_grayscale_image, cmap='gray')
    ax[0].set_title('Original Grayscale Image')
    ax[0].axis('off')

    ax[1].imshow(log_transformed, cmap='gray')
    ax[1].set_title(f'Log Transformation (c={log_scaling_constant})')
    ax[1].axis('off')

    ax[2].imshow(power_law_transformed, cmap='gray')
    ax[2].set_title(f'Power-Law Transformation (c={power_scaling_constant}, gamma={gamma_value})')
    ax[2].axis('off')

    plt.show()



# **Step 1: Upload the grayscale image**

In [28]:
print("Step 1: Please upload a grayscale image.")
upload_grayscale_image()

# Wait for the user to upload an image
while uploaded_grayscale_image is None:
    pass  # Wait for the image to be uploaded

Step 1: Please upload a grayscale image.


FileUpload(value={}, accept='image/*', description='Upload')

Grayscale image uploaded successfully! Please proceed with transformations.


In [29]:
# Step 2: Create sliders for scaling constants and gamma value for transformations
print("Step 2: Adjust the sliders to apply transformations.")
log_scaling_constant_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='c (Log)')
power_scaling_constant_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='c (Power)')
gamma_value_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Gamma')

Step 2: Adjust the sliders to apply transformations.


# **Use interactive widgets to update the display automatically**

In [30]:

ui = widgets.VBox([
    widgets.Label("Adjust the sliders below to apply the transformations:"),
    log_scaling_constant_slider, power_scaling_constant_slider, gamma_value_slider
])
out = widgets.interactive_output(show_transformed_images, {
    'log_scaling_constant': log_scaling_constant_slider,
    'power_scaling_constant': power_scaling_constant_slider,
    'gamma_value': gamma_value_slider
})

# **Display the sliders and the output area**

In [31]:
display(ui, out)


Output()

The main differences between log transformation and power-law (gamma) transformation come from how they change the brightness and contrast in an image. Here’s why they work differently:

### 1. **Log Transformation**
   - **How It Works:** The formula is \( s = c \cdot \log(1 + r) \), where \( c \) is a constant, and \( r \) is the original pixel brightness.
   - **What It Does:** Log transformation brightens the darker parts of the image while toning down the brighter areas. It’s great for showing details in shadows or dark regions.
   - **When It’s Useful:** It works well for images with a lot of dark areas where details are hard to see. It helps bring out those details without making the bright areas too bright.

### 2. **Power-Law (Gamma) Transformation**
   - **How It Works:** The formula is \( s = c \cdot r^\gamma \), where \( c \) is a constant, and \( \gamma \) (gamma) controls how the brightness changes.
   - **What It Does:** You can adjust \( \gamma \) to change how the brightness and contrast look. If \( \gamma < 1 \), it makes dark areas brighter; if \( \gamma > 1 \), it makes bright areas even brighter.
   - **When It’s Useful:** It’s more flexible than log transformation. You can use it to either brighten dark areas or emphasize bright areas, depending on the value of \( \gamma \). This makes it suitable for a wide range of images.

### Why They Look Different:
1. **How They Change Brightness:**
   - **Log Transformation:** Focuses on brightening darker parts, which makes it better at revealing details in shadows. It reduces the intensity of bright areas because it compresses the range of brighter values.
   - **Power-Law Transformation:** Gives more control over which part of the image you want to change. By adjusting \( \gamma \), you can choose to make either dark or bright areas stand out more.

2. **Adjusting Contrast:**
   - **Log Transformation:** Has a fixed way of changing brightness, so you can’t fine-tune it as much.
   - **Power-Law Transformation:** Lets you control the contrast by adjusting \( \gamma \), making it more versatile for enhancing different parts of the image.

3. **Best Uses:**
   - **Log Transformation:** Good for images with lots of dark details that need to be highlighted.
   - **Power-Law Transformation:** Works well for various types of images because you can adjust \( \gamma \) to get the desired effect.

### Conclusion:
- **Log transformation** is great for making dark areas clearer but can make bright areas less intense.
- **Power-law transformation** gives more control over the image’s appearance by adjusting \( \gamma \), allowing you to enhance either dark or bright areas.

The key difference is in how each method changes the brightness, which affects their ability to reveal details and adjust contrast in different parts of the image.